# Hyperelasticity
Author: Jørgen S. Dokken and Garth N. Wells

This section shows how to solve the hyperelasticity problem for deformation of a beam.

We will also show how to create a constant boundary condition for a vector function space.

We start by importing DOLFINx and some additional dependencies.
Then, we create a slender cantilever consisting of hexahedral elements and create the function space `V` for our unknown.

In [16]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
import pyvista
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot
L = 20.0
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, 1, 1]], [20, 5, 5], mesh.CellType.hexahedron)
V = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim, )))

2024-09-16 20:16:40.662 ( 387.134s) [main            ]         graphbuild.cpp:395   INFO| Build local part of mesh dual graph
2024-09-16 20:16:40.663 ( 387.134s) [main            ]           ordering.cpp:203   INFO| GPS pseudo-diameter:(28) 499-0
2024-09-16 20:16:40.663 ( 387.134s) [main            ]           Topology.cpp:901   INFO| Create topology
2024-09-16 20:16:40.663 ( 387.134s) [main            ]                MPI.cpp:166   INFO| Computing communication graph edges (using NBX algorithm). Number of input edges: 1
2024-09-16 20:16:40.663 ( 387.134s) [main            ]                MPI.cpp:237   INFO| Finished graph edge discovery using NBX algorithm. Number of discovered edges 1
2024-09-16 20:16:40.663 ( 387.135s) [main            ]          partition.cpp:233   INFO| Compute ghost indices
2024-09-16 20:16:40.663 ( 387.135s) [main            ]                MPI.cpp:99    INFO| Computing communication graph edges (using PCX algorithm). Number of input edges: 0
2024-09-16 20:16:

We create two python functions for determining the facets to apply boundary conditions to

In [17]:
def left(x):
    return np.isclose(x[0], 0)


def right(x):
    return np.isclose(x[0], L)


fdim = domain.topology.dim - 1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

2024-09-16 20:16:40.690 ( 387.162s) [main            ]topologycomputation.cpp:776   INFO| Requesting connectivity 2 - 3
2024-09-16 20:16:40.690 ( 387.162s) [main            ]topologycomputation.cpp:641   INFO| Computing mesh connectivity 2 - 3 from transpose.
2024-09-16 20:16:40.691 ( 387.162s) [main            ]topologycomputation.cpp:776   INFO| Requesting connectivity 2 - 0
2024-09-16 20:16:40.691 ( 387.162s) [main            ]topologycomputation.cpp:776   INFO| Requesting connectivity 2 - 2
2024-09-16 20:16:40.691 ( 387.162s) [main            ]topologycomputation.cpp:776   INFO| Requesting connectivity 0 - 3
2024-09-16 20:16:40.691 ( 387.162s) [main            ]topologycomputation.cpp:641   INFO| Computing mesh connectivity 0 - 3 from transpose.
2024-09-16 20:16:40.691 ( 387.162s) [main            ]topologycomputation.cpp:776   INFO| Requesting connectivity 3 - 0
2024-09-16 20:16:40.691 ( 387.162s) [main            ]topologycomputation.cpp:776   INFO| Requesting connectivity 2 - 3


Next, we create a  marker based on these two functions

In [18]:
# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

We then create a function for supplying the boundary condition on the left side, which is fixed.

In [19]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=default_scalar_type)

To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [20]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

2024-09-16 20:16:40.734 ( 387.205s) [main            ]topologycomputation.cpp:776   INFO| Requesting connectivity 2 - 3


Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`).

In [21]:
B = fem.Constant(domain, default_scalar_type((0, 0, 0)))
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

Define the test and solution functions on the space $V$

In [22]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [23]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$, and create the expression for the first Piola-Kirchhoff stress:

In [24]:
# Elasticity parameters
E = default_scalar_type(1.0e4)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
P = ufl.diff(psi, F)

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity, the following lines can be uncommented to solve the linear elasticity problem.
```

In [25]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2. We set the quadrature degree for the integrals to 4.

In [26]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)
# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, B) * dx - ufl.inner(v, T) * ds(2)

As the varitional form is non-linear and written on residual form, we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [27]:
problem = NonlinearProblem(F, u, bcs)

2024-09-16 20:16:40.852 ( 387.324s) [main            ]topologycomputation.cpp:776   INFO| Requesting connectivity 2 - 3
2024-09-16 20:16:40.852 ( 387.324s) [main            ]topologycomputation.cpp:776   INFO| Requesting connectivity 3 - 2
2024-09-16 20:16:40.852 ( 387.324s) [main            ]topologycomputation.cpp:776   INFO| Requesting connectivity 2 - 3
2024-09-16 20:16:40.852 ( 387.324s) [main            ]topologycomputation.cpp:776   INFO| Requesting connectivity 3 - 2
2024-09-16 20:16:40.855 ( 387.326s) [main            ]topologycomputation.cpp:776   INFO| Requesting connectivity 2 - 3
2024-09-16 20:16:40.855 ( 387.326s) [main            ]topologycomputation.cpp:776   INFO| Requesting connectivity 3 - 2


and then create and customize the Newton solver

In [28]:
solver = NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"


2024-09-16 20:16:40.869 ( 387.341s) [main            ]    SparsityPattern.cpp:384   INFO| Column ghost size increased from 0 to 0


We create a function to plot the solution at each time step.

In [29]:
from pathlib import Path
current_directory = Path(__file__).resolve().parent
results_folder = Path(str(current_directory) + "/results_hyperelasticity")
results_folder.mkdir(exist_ok=True, parents=True)

pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif(results_folder + "/displacement.gif", fps=3)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.functionspace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([u[i]**2 for i in range(len(u))])), Vs.element.interpolation_points())
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

NameError: name '__file__' is not defined

Finally, we solve the problem over several time steps, updating the z-component of the traction

In [ ]:
log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert (converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    function_grid["u"][:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    warped.points[:, :] = warped_n.points
    warped.point_data["mag"][:] = magnitude.x.array
    plotter.update_scalar_bar_range([0, 10])
    plotter.write_frame()
plotter.close()

2024-09-16 20:10:17.350 (   3.821s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:17.784 (   4.255s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:18.017 (   4.488s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 22.2455 (tol = 1e-08) r (rel) = 0.134278(tol = 1e-08)
2024-09-16 20:10:18.144 (   4.616s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:18.382 (   4.853s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 2.43261 (tol = 1e-08) r (rel) = 0.0146837(tol = 1e-08)
2024-09-16 20:10:18.505 (   4.977s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:18.740 (   5.212s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 

Time step 1, Number of iterations 8, Load [ 0.   0.  -1.5]


2024-09-16 20:10:20.628 (   7.099s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:21.005 (   7.477s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:21.248 (   7.720s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 17.3254 (tol = 1e-08) r (rel) = 0.117842(tol = 1e-08)
2024-09-16 20:10:21.375 (   7.847s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:21.608 (   8.080s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 5.14882 (tol = 1e-08) r (rel) = 0.0350207(tol = 1e-08)
2024-09-16 20:10:21.736 (   8.208s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:21.989 (   8.460s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 

Time step 2, Number of iterations 9, Load [ 0.  0. -3.]


2024-09-16 20:10:24.624 (  11.095s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:24.898 (  11.369s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 10.0011 (tol = 1e-08) r (rel) = 0.0887471(tol = 1e-08)
2024-09-16 20:10:25.030 (  11.502s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:25.276 (  11.747s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 5.33026 (tol = 1e-08) r (rel) = 0.0472992(tol = 1e-08)
2024-09-16 20:10:25.408 (  11.880s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:25.676 (  12.148s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 4: r (abs) = 11.9901 (tol = 1e-08) r (rel) = 0.106397(tol = 1e-08)
2024-09-16 20:10:25.809 (  12.280s) [main            ]              

...

Time step 7, Number of iterations 6, Load [  0.    0.  -10.5]


2024-09-16 20:10:43.196 (  29.668s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:43.475 (  29.946s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 1.06336 (tol = 1e-08) r (rel) = 0.031085(tol = 1e-08)
2024-09-16 20:10:43.609 (  30.081s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:43.845 (  30.316s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 2.0477 (tol = 1e-08) r (rel) = 0.0598598(tol = 1e-08)
2024-09-16 20:10:43.978 (  30.450s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:44.311 (  30.782s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 4: r (abs) = 0.00897719 (tol = 1e-08) r (rel) = 0.000262427(tol = 1e-08)
2024-09-16 20:10:44.457 (  30.929s) [main            ]          

Time step 8, Number of iterations 6, Load [  0.   0. -12.]


2024-09-16 20:10:45.735 (  32.207s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:45.996 (  32.467s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 0.898789 (tol = 1e-08) r (rel) = 0.0309666(tol = 1e-08)
2024-09-16 20:10:46.121 (  32.592s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:46.387 (  32.859s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 1.38354 (tol = 1e-08) r (rel) = 0.0476679(tol = 1e-08)
2024-09-16 20:10:46.511 (  32.983s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-16 20:10:46.913 (  33.385s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 4: r (abs) = 0.00185096 (tol = 1e-08) r (rel) = 6.37724e-05(tol = 1e-08)
2024-09-16 20:10:47.083 (  33.555s) [main            ]       

Time step 9, Number of iterations 6, Load [  0.    0.  -13.5]


<img src="./results_hyperelasticity/deformation.gif" alt="gif" class="bg-primary mb-1" width="800px">